# Project

## Part - 1 - Domain Model Definition

### Download Data

In [ ]:
import urllib
import json
import pandas as pd
from code.interpro_data import *
from code.HmmPy import *
from code.metrics import *
from Bio import SearchIO, SeqIO

url = "https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam/pf03060?format=json"

In [ ]:
our_domain = 'KTFEVRYPIIQAPMAGASTLELAATVTRLGGIGSIPMGSLSEKCDAIETQLENFDELVGDSGRIVNLNFFAHKEPRSGRADVNEEWLKKYDKIYGKAGIEFDKKELKLLYPSFRSIVDPQHPTVRLLKNLKPKIVSFHFGLPHEAVIESLQASDIKIFVTVTNLQEFQQAYESKLDGVVLQGWEAGGHRGNFKANDVEDGQLKTLDLVSTIVDYIDSASISNPPFIIAAGGIHDDESIKELLQFNIAAVQLGTVWLPSSQATISPEHLKMFQSPKSDTMMTAAISGRNLRTISTPFLRDLHQSSPLASIPDYPLPYDSFKSLANDAKQSGKGPQYSAFLAGSNYHKSWKDTRSTEEIFSILVQ'
print(len(our_domain))
print(our_domain[9:372])

In [ ]:
metadata, entries, gt = get_data(url, 1)
print(gt.shape)
gt.head(10)

Here we download with the InterPro APIs our ground truth for our domain. We get 23 sequences in the Interpro database that match our PFAM family.

### Model Creation
To build our models, our first task was to retrieve sequence homologues with our domain sequence.
To do this, we performed both searches in reviewed and unreviewed databases, and we kept the significant hits (E-value < 0.01). We then made some attempts on varying the number of significant hits to see if that could improve the performances of the models.

- In Swiss-Prot (annotated) we found 14 significant hits; (Call those try_1)
- In Uniref90 we found 250 significant hits. (try_3)
- From the 250 significant hits from Uniref90, we tried to take just the first 100 (try_4)

For each of those sets of retrieved hits, we performed a multiple sequence alignment (MSA). To do this, made the MSAs using three different algorithms: T-Coffee, Muscle and Clustal-Omega. Comparing the results we found that T-Coffee is generally better.

For each of those MSAs, we built a PSSM (Position-Specific Scoring Matrix) and a HMM (Hidden Markov Model). We then performed a PSI-BLAST search and HMM-Search using respectively our PSSM and our HMM as inputs. To do this we used the command line versions of *blast+* and *hmmer*; the bash scripts that manage the model creation can be found in data\PSSMs\pssm+psiblast.sh  data\HMMSs\generate_hmms.sh 


### Evaluate Ability of Matching sequences
Once we got the hits from our models, using PSI-BLAST and HMMER searches, we parsed the output files inside tables, which contain, for each hit:
- The Protein UniProt ID;
- The relative E-value of the hit;
- Where the model predicted the domain to begin;
- Where the model predicted the domain to end;

At this point, to evaluate the ability of our model to find the right sequences, we had to check how many sequences the model:
- Correctly predicted to contain the domain (TP);
- Wrongly predicted to contain the domain (FP);
- Correctly predicted **not** to contain the domain (TN);
- Wrongly predicted **not** to contain the domain (FN).

In [ ]:
# the last 3 parameters are thresholds for the E-values. Our models will consider significant only sequences that have e-values under that threshold.

# - the first one is the Sequence E-Value for HMMs;
# - the second one is the Domain independent E-value for HMMs;
# - the third one is the E-Value returned by the Psi-Blast searches made with our PSSM models.

threshold_hmms_e_value = 0.01
threshold_hmms_i_e_value = 0.01
threshold_pssm_e_value = 0.01

In [ ]:
metrics_df, parsed_tblouts, parsed_domtblouts, parsed_psiblast = metrics_8(gt, threshold_hmms_e_value, threshold_hmms_i_e_value, threshold_pssm_e_value)
metrics_df

Since we have an highly unbalanced dataset, some of those metrics will be useless to us. We restrict just on the useful metrics, that in this case are precision, MCC and F1-Score.
Maybe the most important in our case is precision, since we found that our models thend to have almost perfect recall, but bad precision. So we sort the data by precision to see the best models.

In [ ]:
metrics_df_useful = metrics_df.loc[:,('n_hits','precision','recall', 'mcc','f1_score')]
metrics_df_useful.sort_values(by='precision', axis=0, ascending=False).head(5)

We see a high number of hits, which bring our precision down, but the recall is always high, which suggests us that the model is finding almost all the ground truth proteins, but also finds some other proteins that are negative.
We try to plce a even lower threshold on the E-value to see if the performances are better.

In [ ]:
threshold_hmms_e_value = 0.001
threshold_hmms_i_e_value = 0.001
threshold_pssm_e_value = 0.001

In [ ]:
metrics_df2, parsed_tblouts2, parsed_domtblouts2, parsed_psiblast2 = metrics_8(gt, threshold_hmms_e_value, threshold_hmms_i_e_value, threshold_pssm_e_value)

In [ ]:
metrics_df2
metrics_df2_useful = metrics_df2.loc[:,('n_hits','precision','recall', 'mcc','f1_score')]
summary8 = metrics_df2_useful.sort_values(by='precision', axis=0, ascending=False).head(5)
summary8

#### Plotting Metrics

In [ ]:
plot_metrics(summary8)

In [ ]:
plot_metrics_models(summary8, 9)

In [ ]:
plot_metrics_summary(summary8, 8, threshold_hmms_e_value, threshold_hmms_i_e_value, threshold_pssm_e_value)

### Evaluate the ability of matching domain positions
Here we want to evaluate how good our model is at estimating domain positions inside the found proteins.

We computed in a similar fashion, the values to obtain the metrics also in this case. 

Note that, technically, we should consider as True Negatives also all the proteins in SwissProt that our model correctly predicted not to have the domain (that is, all the proteins that our model didn't find, and that actually don't have the domain). While this is true, in this way any model would have an extremely high number of True Negatives, since the number of significant hits of a model is always much smaller than the size of uniprot (around 560.000 sequences). This statistic would not help us in any way to assess the ability of our models, and would significantly slow our computation times, so we decided not to compute that quantity.
Below we report our results with some of the better performing models.

In [ ]:
threshold_hmms_e_value = 0.01
threshold_hmms_i_e_value = 0.01
threshold_pssm_e_value = 0.01

In [ ]:
metrics_df, conf_df = metrics_9(parsed_domtblouts, parsed_psiblast, gt, threshold_hmms_e_value, threshold_hmms_i_e_value, threshold_pssm_e_value)
metrics_df

In [ ]:
metrics_df_useful = metrics_df.loc[:,('precision','recall', 'mcc','f1_score')]
metrics_df_useful.sort_values(by='precision', axis=0, ascending=False).head(5)

In [ ]:
threshold_hmms_e_value = 0.001
threshold_hmms_i_e_value = 0.001
threshold_pssm_e_value = 0.001

In [ ]:
metrics_df2, conf_df2 = metrics_9(parsed_domtblouts, parsed_psiblast, gt, threshold_hmms_e_value, threshold_hmms_i_e_value, threshold_pssm_e_value)

In [ ]:
metrics_df2
metrics_df2_useful = metrics_df2.loc[:,('precision','recall', 'mcc','f1_score')]
summary = metrics_df2_useful.sort_values(by='precision', axis=0, ascending=False).head(5)
summary

#### Plotting Metrics

In [ ]:
plot_metrics(summary)

In [ ]:
plot_metrics_models(summary, 9)

In [ ]:
plot_metrics_summary(summary, 9, threshold_hmms_e_value, threshold_hmms_i_e_value, threshold_pssm_e_value)

## Part - 2 - Domain Family Characterization
From this point, we will have to choose a single best model. 
Since we will probably struggle to find the best model until the end, here the "PATH_MODEL_PROTS" will be the path of the csv file containing the prediction of the future best model.

out_psiblast_M_4_denoised1_uniref90_1iterations.xml is the **final best model**.

In [ ]:
import urllib
import json
import pandas as pd
from code.interpro_data import *
from code.HmmPy import *
from code.metrics import *
from Bio import SearchIO, SeqIO
from Bio.PDB.PDBList import *
from os import path

In [ ]:
from code.family_structures import *
bestmodel = 'psiblast_M_4_denoised1_uniref90_1iterations'
letter = bestmodel.split('_')[1]
# original dataset
PATH_MODEL_PROTS = path.join('data', 'part_1', 'PSSMs', 'PSSM_{}'.format(letter), 'parsed', 'out_{}.csv'.format(bestmodel)) #'.\data_team_1\PSSMs\PSSM_C\parsed\out_{}.csv'.format(bestmodel)

# map from pdb chains to uniprot entries
# use your path where you saved the file for now
# SIFTS_PATH = '..\midterm exams\midterm2\data\pdb_chain_uniprot.tsv'
SIFTS_PATH = path.join('data', 'part_2', 'original_datasets', 'uniprot_segments_observed.tsv') #'data_team_1\\_part_2\\original_datasets\\uniprot_segments_observed.tsv'

In [ ]:
## DONT DELETE THIS CELL. We use it to print the proteins found by the best model for ease of copy-pasting
# the uniprot codes in https://www.uniprot.org/uploadlists/ to create "family_sequences" database!

model_prots_df = pd.read_csv(PATH_MODEL_PROTS)
model_prots = list(model_prots_df.ids.values)
for i in model_prots[:5]:
    print(i)

In [ ]:
pdb_db = generate_pdb_df(SIFTS_PATH, PATH_MODEL_PROTS)
# pdb_db.to_csv(".\\data_team_1\\_part_2\\mappings\\mapping_{}.csv".format(bestmodel))
pdb_db.to_csv(path.join('data', 'part_2', 'mappings', 'mapping_{}.csv'.format(bestmodel)))
pdb_db.head()

In [ ]:
# keep only chains with 80% or more coverage
pdb_db_filtered = filter_pdb_db(pdb_db)
pdb_ids = list(pdb_db_filtered.pdb.unique())
# print(pdb_ids)
print(len(pdb_ids))

In [ ]:
# Print for copy -paste into PDB website
for i in pdb_ids:
    print(i)

In [ ]:
# #We use the function "retrieve_pdb_file" provided by Biopython to automatically download all the needed pdb files.
# pdblist = PDBList(server='ftp://ftp.wwpdb.org')
# pdblist.download_pdb_files(pdb_codes = [code.upper() for code in pdb_ids], pdir = '.\\data_team_1\\_part_2\\original_datasets\\family_structures\\pdbs_{}'.format(bestmodel), file_format="pdb")

### Structural Characterization

#### Pairwise Structural Alignment between all the pdbs.

**This must be made in linux**

To make them in a procedural way we use the TMAlign command line program.
Specifically, we generate all the .out files with this bash script:

```
echo "Enter model type (psiblast or hmm)"
read model

echo "If PSIBLAST, how many iterations{"
read iterations


echo "Enter MSA method (C, M or O)"
read msamethod

echo "Enter try number"
read try

echo "Enter database (swissprot, uniref90, uniref50 or uniref100)"
read db

if [ $model == 'psiblast' ]
then

directoryname=pdbs_${model}_${msamethod}_${try}_${db}_${iterations}iterations
else
directoryname=pdbs_${model}_${msamethod}_${try}_${db}
fi

for ent1 in ./${directoryname}/*.ent; do
	for ent2 in ./${directoryname}/*.ent; do
		#echo "${ent1}_${ent2}"
		TMalign ${ent1} ${ent1} > ./temp/$(basename ${ent1})_$(basename ${ent2}).out
	done
done



In [ ]:
# Once the .out files are done we need to parse them.
# Since the generate .out files are a huge amount of files will result in a lot of used disk space, since all the info we need in those files is the TM-Scores and the RMSD, we will delete them after we finished to read them.
rmsdmatrix = create_rmsd_matrix(bestmodel)
rmsdmatrix

In [ ]:
tmscorematrix = create_tmscores_matrix(bestmodel)
tmscorematrix

In [ ]:
# DELETE THE .ent FILES, NOW THAT WE FINISHED READING THEM
clear_temp_folder()

#### Visualizing the Matrices
Here we visualize both Matrices with the help of a heatmap, to help us spot any peculiar pattern or potential outliers.

In [ ]:
# csvs_path = "./data_team_1/_part_2/original_datasets/family_structures/pdbs_{}/".format(bestmodel)
csvs_path = path.join('data', 'part_2', 'original_datasets', 'family_structures', 'pdbs_{}'.format(bestmodel))
rmsd_filename = "rmsds_{}".format(bestmodel)
tmscore_filename = "tmscores_{}".format(bestmodel)

In [ ]:
plot_heatmap(csvs_path, rmsd_filename, header='RMSD Matrix')

In [ ]:
plot_heatmap(csvs_path, tmscore_filename, header='TM-Score Matrix')

#### Plot Dendograms

In [ ]:
# plot_dendogram(rmsdmatrix, header = "Hierarchical clustering using rmsd", save_path = csvs_path + 'dendo_rmsd.png')
plot_dendogram(rmsdmatrix, header = "Hierarchical clustering using rmsd", save_path = path.join(csvs_path, 'dendo_rmsd.png'))

In [ ]:
# plot_dendogram(tmscorematrix, header = "Hierarchical clustering using TM-Score", save_path = csvs_path + 'dendo_tmscore.png' )
plot_dendogram(tmscorematrix, header = "Hierarchical clustering using TM-Score", save_path = path.join(csvs_path, 'dendo_tmscore.png'))

#### Remove 2 outliers 
From the heatmaps and the dendograms we can easily spot two outliers.
We Remove 6u8n and 6udq from the analisys.

In [ ]:
rmsdmatrix_2 = rmsdmatrix.drop(['6u8n','6udq'], axis = 0)
rmsdmatrix_2 = rmsdmatrix_2.drop(['6u8n','6udq'], axis = 1)
print(rmsdmatrix_2.shape)
tmscorematrix_2 = tmscorematrix.drop(['6u8n','6udq'], axis = 0)
tmscorematrix_2 = tmscorematrix_2.drop(['6u8n','6udq'], axis = 1)
rmsdmatrix_2.shape

In [ ]:
# replot dendograms and heatmaps
# rmsdmatrix_2.to_csv("./data_team_1/_part_2/original_datasets/family_structures/pdbs_{}/rmsds2_{}.csv".format(bestmodel, bestmodel))
rmsdmatrix_2.to_csv(path.join('data', 'part_2', 'original_datasets', 'family_structures', 'pdbs_{}'.format(bestmodel), 'rmsds2_{}.csv'.format(bestmodel)))

# tmscorematrix_2.to_csv("./data_team_1/_part_2/original_datasets/family_structures/pdbs_{}/tmscores2_{}.csv".format(bestmodel, bestmodel))
tmscorematrix_2.to_csv(path.join('data', 'part_2', 'original_datasets', 'family_structures', 'pdbs_{}'.format(bestmodel), 'tmscores2_{}.csv'.format(bestmodel)))

In [ ]:
plot_heatmap(csvs_path, 'rmsds2_{}'.format(bestmodel), header='RMSD Matrix (No Outliers)')

In [ ]:
plot_heatmap(csvs_path, 'tmscores2_{}'.format(bestmodel), header='TM-Scores Matrix (No Outliers)')

In [ ]:
# plot_dendogram(rmsdmatrix_2, header = "Hierarchical clustering using RMSD (No Outliers)", save_path = csvs_path + 'dendo_rmsd_no_outliers.png')
plot_dendogram(rmsdmatrix_2, header = "Hierarchical clustering using RMSD (No Outliers)", save_path = path.join(csvs_path, 'dendo_rmsd_no_outliers.png'))

In [ ]:
# plot_dendogram(tmscorematrix_2, header = "Hierarchical clustering using TM-Score (No Outliers)", save_path = csvs_path + 'dendo_tmscore_no_outliers.png')
plot_dendogram(tmscorematrix_2, header = "Hierarchical clustering using TM-Score (No Outliers)", save_path = path.join(csvs_path, 'dendo_tmscore_no_outliers.png'))

#### mTM-Align
We now perform multiple structural alignment on our *family_structures* dataset (without the outliers!) to identify and visualize conserved positions.



In [ ]:
from Bio.PDB import PDBList, is_aa, PDBIO
from Bio.PDB.PDBParser import PDBParser
from Bio.SeqUtils import IUPACData
from Bio.PDB.PDBIO import Select
from Bio.SeqIO.PdbIO import PdbSeqresIterator

from Bio.PDB import PDBList, NeighborSearch
from Bio.PDB.PDBParser import PDBParser
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def get_distance_matrix(residues, threshold=False, seq_sep=6):

    # Calculate the distance matrix
    distances = []
    for residue1 in residues:
        if residue1.id[0] == " ":  # Exclude hetero/water residues
            row = []
            for residue2 in residues:
                if residue2.id[0] == " ":  # Exclude hetero/water residues
                    if abs(residue1.id[1] - residue2.id[1]) >= seq_sep:
                        if threshold: #this way we can compute the distance matrices without 
                            #taking into account the threshold (so to answer question 5 
                            # independentely of question 6)
                            if (residue1["CA"] - residue2["CA"]) <= threshold:
                                row.append(residue1["CA"] - residue2["CA"])
                            else:
                                row.append(None)
                        else:
                            row.append(residue1["CA"] - residue2["CA"])
                    else:
                        row.append(None)
            distances.append(row)

    return np.array(distances, dtype=float)

In [ ]:
# path_struct = '.\\data_team_1\\_part_2\\original_datasets\\family_structures\\pdbs_{}\\superimposed_structure_core.pdb'.format(bestmodel)
# path_struct_all = '.\\data_team_1\\_part_2\\original_datasets\\family_structures\\pdbs_{}\\superimposed_structure_all.pdb'.format(bestmodel)
path_struct = path.join('data', 'part_2', 'original_datasets', 'family_structures', 'pdbs_{}'.format(bestmodel), 'superimposed_structure_core.pdb')
path_struct_all = path.join('data', 'part_2', 'original_datasets', 'family_structures', 'pdbs_{}'.format(bestmodel), 'superimposed_structure_all.pdb')

structure = PDBParser(QUIET=True).get_structure('core', path_struct)
selected_residues = structure[0]

In [ ]:
sequence_separation = 12
threshold = False
dist_matrix = [get_distance_matrix(residue, threshold, sequence_separation) for residue in selected_residues]
# dist_matrix = get_distance_matrix(selected_residues['A'], threshold, sequence_separation) 

In [ ]:
ncols = 5
nrows = int(np.ceil(len(dist_matrix) / ncols))

plt.figure(figsize = (20,15))
fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize=(3*ncols, 2*nrows))
plt.subplots_adjust(wspace=.05, hspace=.1)

for i, dm in enumerate(dist_matrix):
    sns.heatmap(dist_matrix[i], ax=ax[i//ncols, i%ncols], cmap='Blues')#, linewidths=.01, linecolor="black")

#plt.savefig("data_team_1\\_part_2\\original_datasets\\family_structures\\pdbs_{}\\dist_matrices_{}.pdf".format(bestmodel, bestmodel), bbox_inches='tight')
plt.savefig(path.join('data', 'part_2', 'original_datasets', 'family_structures', 'pdbs_{}'.format(bestmodel), 'dist_matrices_{}.pdf'.format(bestmodel)), bbox_inches='tight')
plt.show()

#### Long Range Contacts
In this section we want to visualize which residues are close in the structural sequence alignment but far (more than 12 residues apart) in the sequence.

In [ ]:
sequence_separation = 12
threshold = 8
contact_matrix = [get_distance_matrix(residue, threshold, sequence_separation) for residue in selected_residues]

In [ ]:
contact_matrix = [np.nan_to_num(x, nan = 0) for x in contact_matrix]
avg_contact_matrix = np.mean(contact_matrix, axis=0)

In [ ]:
_ = sns.heatmap(avg_contact_matrix, cmap='Blues')

# plt.savefig("data_team_1\\_part_2\\original_datasets\\family_structures\\contact_map.pdf", bbox_inches='tight')
plt.savefig(path.join('data', 'part_2', 'original_datasets', 'family_structures', 'contact_map.pdf'), bbox_inches='tight')

#### CATH Superfamily & Family
Download file .tsv file here and place it in the original_dataset folder (I already added it to the .gitignore)
ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/tsv/pdb_chain_cath_uniprot.tsv.gz

We also use the cath_b.20201021.tsv file to map CATH_ids to CATH superfamilies.

To make this point we will just need to process this .tsv similarly as we did to build our family_structures dataset.

In [ ]:
# all our proteins
print(model_prots)
print()
print(len(model_prots))

In [ ]:
# uniprot_cath_path = '.\\data_team_1\\_part_2\\original_datasets\\pdb_chain_cath_uniprot.tsv'
uniprot_cath_path = path.join('data', 'part_2', 'original_datasets', 'pdb_chain_cath_uniprot.tsv')
uniprot_cath_map = pd.read_csv(uniprot_cath_path, sep = '\t', header=1)
uniprot_cath_map.head()

In [ ]:
# cath_superf_path = '.\\data_team_1\\_part_2\\original_datasets\\cath_b.20201021.tsv'
cath_superf_path = path.join('data', 'part_2', 'original_datasets', 'cath_b.20201021.tsv')
cath_superf_map = pd.read_csv(cath_superf_path, sep = ' ', header=None)
cath_superf_map = cath_superf_map.drop(1, axis=1)
cath_superf_map.columns = ['CATH_ID', 'SUPERF_ID', 'POS (?)']
cath_superf_map.head()

In [ ]:
uniprot_cath_map_model = uniprot_cath_map[uniprot_cath_map.SP_PRIMARY.isin(model_prots)].reset_index()
uniprot_cath_map_model.head()

In [ ]:
model_superfs = cath_superf_map[cath_superf_map.CATH_ID.isin(uniprot_cath_map_model.CATH_ID)].SUPERF_ID
print(model_superfs.values)

In [ ]:
uniprot_cath_map_model['CATH_SUPERFAMILY'] = model_superfs.values
uniprot_cath_map_model.head()

In [ ]:
plt.hist(uniprot_cath_map_model.CATH_SUPERFAMILY)
plt.show()
# http://www.cathdb.info/version/latest/superfamily/3.20.20.70

### Taxonomy

#### Retrieve Taxonomy Lineage

In [ ]:
import pandas as pd
from os import path
import numpy as np
from Bio import SeqIO, SearchIO

How to obtain family_seqs_tax_data:

search all protein ids obtained with the best model on https://www.uniprot.org/uploadlists/ -> select FROM UniprotKB TO uniref90. then download .fasta format, parse it in order to obtain uniref ids -> remove "UniRef90_" chars and search all list on UniProt. Add "tax_lineage", "tax_id" columns and download .tsv file.

In [ ]:
p = path.join('data', 'part_2', 'taxonomy', 'Uniprot_to_Uniref90.fasta')

In [ ]:
l = list(SeqIO.parse(p, "fasta"))
for i in l:
    print(i.id[9:])

In [ ]:
tax_data_path = path.join('data', 'part_2', 'taxonomy', 'family_seqs_tax_data.tab')

tax_data_db = pd.read_csv(tax_data_path, sep='\t')

tax_data_db = tax_data_db.drop(['Status','Organism ID', 'Entry name','Gene names', 'Length','Protein names'], axis=1)
tax_data_db.columns = ['uniprot_id', 'organism', 'tax_id', 'tax_lineage']
tax_data_db.head()

In [ ]:
# Save TAX IDs to file
np.savetxt(path.join('data', 'part_2', 'original_datasets', 'tax_ids_{}.txt'.format(bestmodel)) , X = tax_data_db.tax_id.values, fmt = '%d')

In [ ]:
#create new dict {key: P37527 ,value: 'cellular organisms',
#                                        'Bacteria',
#                                        'Terrabacteria group',
#                                        'Firmicutes',
#                                        'Bacilli',
#                                        'Bacillales',
#                                        'Bacillaceae',
#                                        'Bacillus',
#                                        'Bacillus subtilis group',
#                                        'Bacillus subtilis',
#                                        'Bacillus subtilis subsp. subtilis',
#                                        'Bacillus subtilis (strain 168)'}
lineage = {}
for i in range(tax_data_db.shape[0]):
    #print(tax_data_db["uniprot_id"][i])
    lineage.setdefault(tax_data_db["uniprot_id"][i], tax_data_db["tax_lineage"][i].split(","))

In [ ]:
#show example
lineage["P12269"]

In [ ]:
#create frequency dict (key: 'cellular organisms' , value :100)

freq_lineage = {}
for prot, tax in lineage.items():
    for elem in tax:
        freq_lineage[elem]=freq_lineage.setdefault(elem,0)+1

#### Plot Taxonomic Tree

Install ```ete3``` with ```pip``` but be aware that it may not install all the dependencies required for the package to run. To solve the issue locate the ```__init__.py``` of the library, change all the ```pass``` into ```raise``` and find out which module you are missing.

In [ ]:
from ete3 import Tree, TreeStyle, TextFace, NodeStyle, faces, AttrFace, CircleFace

How to download phyliptree.phy:

go to https://www.ncbi.nlm.nih.gov/Taxonomy/CommonTree/wwwcmt.cgi, and upload the previously saved .txt file (e.g. tax_ids_psiblast_M_4_denoised1_uniref90_1iterations.txt)

In [ ]:
tree=Tree(path.join('data', 'part_2', 'taxonomy', 'phyliptree.phy'),format=1,quoted_node_names=True)

In [ ]:
def layout(node):
    if node.is_leaf():
        # Add node name to leaf nodes
        N = AttrFace("name", fsize=35, fgcolor="black")
        faces.add_face_to_node(N, node, 0)
    if "weight" in node.features:
        # Creates a sphere face whose size is proportional to node's
        # feature "weight"
        C = CircleFace(radius=node.weight/2, color="Crimson", style="circle")
        # Let's make the sphere transparent
        C.opacity = 0.5
        # And place as a float face over the tree
        faces.add_face_to_node(C, node, 0, position="float")

for i, n in enumerate(tree.traverse()): #add freq to each node to show a abundance
    w = [value for key,value in freq_lineage.items()][i]
    n.add_features(weight=w)

ts = TreeStyle()
ts.title.add_face(TextFace("Taxonomic Tree", fsize=90), column=0)
ts.scale =  100
ts.branch_vertical_margin = 10
ts.layout_fn = layout

# Draw a tree
ts.mode = "r"

# We will add node names manually
ts.show_leaf_name = False
# Show branch data
ts.show_branch_length = False
ts.show_branch_support = False
#tree.show(tree_style=ts)
tree.render(path.join('data', 'part_2', 'taxonomy', 'taxonomy_t_{}.png'.format(bestmodel)), w= 250 ,h=250, units="mm",tree_style=ts,dpi=600);

### Functional Analysis

In [ ]:
import gzip
import copy
from Bio import SeqIO, SearchIO
from code import parse_go_obo
import pandas as pd
import matplotlib.pyplot as plt
from os import path

# Paths of the gene ontology file, All GOAs for the Swissprot databas
go_path = path.join('data', 'part_2', 'original_datasets', 'go.obo')
goa_path = path.join('data', 'part_2', 'original_datasets', 'swissprot_goa_all.goa') 
family_sequences_path = path.join('data', 'part_2', 'original_datasets', 'family_sequences', 'family_seqs_{}.fasta'.format(bestmodel))

family_sequences = list(SeqIO.parse(family_sequences_path, "fasta"))

In [ ]:
# I now parse the whole ontology file
# dictionary with keys: proteins, items: ontology terms associated to the protein
graph = parse_go_obo.parse_obo(go_path)
ancestors, depth, roots = parse_go_obo.get_ancestors(graph)
children = parse_go_obo.get_children(ancestors)

In [ ]:
# create the dataframe for the whole Swissprot GOA
goa_db = pd.read_csv(goa_path, sep='\t')
goa_db = goa_db.loc[:,('Entry', 'Gene ontology IDs')]
goa_db.columns = ['uniprot_id', 'go_terms']
goa_db.go_terms =goa_db.go_terms.map(lambda x: x.split("; "))
goa_db['n_go_terms'] = goa_db.go_terms.map(lambda x: len(x))
print('goa_db_shape: {}'.format(goa_db.shape))
goa_db.head()

In [ ]:
family_sequences_ids = [seq.id[9:] for seq in family_sequences]
print('Number of proteins in our family_sequences database: {}'.format(len(family_sequences_ids)))

In [ ]:
# We now take only the proteins that are in our family_sequences 
goa_db_model = goa_db[goa_db.uniprot_id.isin(family_sequences_ids)]
goa_db_model = goa_db_model.reset_index(drop=True)
goa_db_model['n_go_terms'] = goa_db_model.go_terms.map(lambda x: len(x))
print('goa_db_model shape: {}'.format(goa_db_model.shape))
goa_db_model.head()

In [ ]:
# How many repeated/unique, direct annotations in our dataset?
repeated_goas_dataset = pd.Series(goa_db_model.go_terms.sum())
unique_goas_dataset = repeated_goas_dataset.unique()

print('Total number of GOAs: {}'.format(len(repeated_goas_dataset)))
print('Number of unique GOAs: {}'.format(len(unique_goas_dataset)))

In [ ]:
# How many repeated/unique annotations in our dataset, after considering ancestors?
# To see this we create a new dataframe where the go_terms will now include also all the ancestors of all the terms
def include_ancestors(terms_list):
    """
    Given a list of GO terms, returns a list of GO terms where all the ancestors of each term in terms_list is included. NB: the ancestors are added with repetition. """
    terms_list_ancestors = terms_list.copy()
    for term in terms_list:
        if term in ancestors.keys():
            terms_list_ancestors.extend(list(ancestors[term]))
    
    return list(set(terms_list_ancestors))

In [ ]:
# Create Database with goas of our model, including ancestors
goa_db_model_anc = goa_db_model.copy()
goa_db_model_anc.go_terms = goa_db_model.go_terms.map(lambda x: include_ancestors(x))
goa_db_model_anc['n_go_terms'] = goa_db_model_anc.go_terms.map(lambda x: len(x))
print("goa_db_model_anc shape: {}".format(goa_db_model_anc.shape))
goa_db_model_anc.head()

In [ ]:
# # How many repeated/unique annotations in our dataset, after considering ancestors?
repeated_goas_dataset_anc = pd.Series(goa_db_model_anc.go_terms.sum())
unique_goas_dataset_anc = repeated_goas_dataset_anc.unique()

print('Total number of GOAs: {}'.format(len(repeated_goas_dataset_anc)))
print('Number of unique GOAs: {}'.format(len(unique_goas_dataset_anc)))

To find the enriched terms we have to methods:
- Compute fold increase, and see the terms that have fold increase > 1;
- Compute left and right p-values with fischer exact test and see the ones that have right-p value almost 0 and left p value almost 1;

For both those two methods we will have to build a confusion matrix for each GO term, as defined below.

The fold increase can be then calculated dividing the ratio having-/not-having the property of the selected with the ratio having-/not-having of the not selected.

In [ ]:
# Confusion matrix for each term must be like this.
#                 |  Having the property      | Not having the property |
# Selected        |           (1)             |         (2)             |
# --------------- |---------------------------|-------------------------|
# Not selected    |____________(3)____________|__________(4)____________|


# 1 = N. of proteins with GO_i in our dataset;
# 2 = N. of proteins without GO_i in our dataset;
# 3 = N. of proteins with GO_i outside our dataset;
# 4 = N. of proteins without GO_i ourtside our dataset;

# Fold increase = (1/2) / (3/4)

In [ ]:
# We create two dictionaries that will keep the count of how many proteins inside, and outside our dataset, have the GO annotation used as key.

proteins_count_dataset_anc = {}  # { term : count } count within our dataset
proteins_count_rest_anc = {}  # { term : count } count outsite our dataset

# convert to dictionary our dataframe so we can iterate on it
df_dict = goa_db.set_index('uniprot_id').to_dict()['go_terms']
for j, (acc, annotations) in enumerate(df_dict.items()):
    # print(acc)
    # print(annotations)
    terms1 = annotations.copy()
    terms = include_ancestors(terms1)

    # now populate the dictionaries
    if acc in family_sequences_ids:
        for term in terms:
            proteins_count_dataset_anc.setdefault(term, 0)
            proteins_count_dataset_anc[term] += 1
    else:
        for term in terms:
            proteins_count_rest_anc.setdefault(term, 0)
            proteins_count_rest_anc[term] += 1

In [ ]:
print("Number of unique terms outside dataset, considering ancestors: {}".format(len(proteins_count_rest_anc)))

print("Number of unique terms inside dataset, considering ancestors: {}".format(len(proteins_count_dataset_anc)))


In [ ]:
# prof solution
data = []
proteins_rest = goa_db.shape[0] - len(family_sequences_ids)

for term in proteins_count_dataset_anc:
    ratio_set = (proteins_count_dataset_anc[term] + 1) / (len(family_sequences_ids) -
        proteins_count_dataset_anc[term] + 1)  # add pseudo count
    ratio_rest = proteins_count_rest_anc.get(term, 1) / (proteins_rest -
        proteins_count_rest_anc.get(term, 0) + 1)  # add pseudo count
    fold_increase = ratio_set / ratio_rest
    data.append((term, fold_increase))

In [ ]:
data[:5]

In [ ]:
data_filter = [item for item in data if item[1]>1]
data_filter[:5]

In [ ]:
print(len(data))
print(len(data_filter))

In [ ]:
enriched_terms = [el[0] for el in data_filter]
print(enriched_terms[:10])

In [ ]:
enriched_terms_desc = [graph[enriched_term]['def'] for enriched_term in enriched_terms]

In [ ]:
dataset = model_prots

conf_tables = {} # {term : table}
for term in proteins_count_dataset_anc.keys():
    if term in proteins_count_dataset_anc.keys():
        with_goi_dataset = proteins_count_dataset_anc[term]
        without_goi_dataset = len(dataset) - with_goi_dataset
    else:
        with_goi_dataset = 0
        without_goi_dataset  = len(dataset)
    if term in proteins_count_rest_anc.keys():
        with_goi_rest = proteins_count_rest_anc[term]
        without_goi_rest = proteins_rest - with_goi_rest
    else:
        with_goi_rest = 0
        without_goi_rest = proteins_rest

    conf_tables.setdefault(term, [])
    conf_tables[term] = np.array([[with_goi_dataset, without_goi_dataset], [with_goi_rest,without_goi_rest]])

In [ ]:
from scipy import stats
pvals = {} # {term : pvalues}
for term in conf_tables:
    conf_table = conf_tables[term]
    _, twosided_p = stats.fisher_exact(conf_table)
    _, left_p = stats.fisher_exact(conf_table, alternative='less')
    _, right_p = stats.fisher_exact(conf_table, alternative='greater')
    pvals.setdefault(term, [])
    pvals[term] = [twosided_p, left_p, right_p]

In [ ]:
pvals = pd.DataFrame.from_dict(pvals, columns=['twosided', 'left', 'right'], orient='index').reset_index()
pvals

In [ ]:
pvals_index = pvals['index'].to_list()
data_dict = {}

for dat in data:
    data_dict[dat[0]] = dat[1]

ordered_data = []

for ind in pvals_index:
    ordered_data.append(data_dict[ind])
    
pvals['fold_increase'] = ordered_data

In [ ]:
pvals_sorted = pvals.sort_values(by=['fold_increase'], ascending=False).reset_index(drop=True)

In [ ]:
n_children = {}

for i, (k, v) in enumerate(children.items()):
    if k in pvals_index:
        n_children[k] = len(v)

pvals_sorted['n_children'] = pvals_sorted['index'].apply(lambda x: n_children[x] if x in list(n_children.keys()) else 0)

In [ ]:
descriptions = {}

for i, (k, v) in enumerate(graph.items()):
    if k in pvals_index:
        descriptions[k] = v['def']

pvals_sorted['description'] = pvals_sorted['index'].apply(lambda x: descriptions[x] if x in list(descriptions.keys()) else '')
pvals_sorted['description_red'] = pvals_sorted.description.map(lambda x: " ".join(x.split(' ')[:3] + ['...']) if len(x.split(' '))>3 else x )

In [ ]:
pvals_sorted.sort_values(by=['fold_increase'], ascending=False).head(10)

In [ ]:
pvals_sorted[pvals_sorted.loc[:,'index'] == 'GO:0018580'].description_red.values[0]

#### Wordcloud

In [ ]:
from wordcloud import (WordCloud, get_single_color_func)
import matplotlib.pyplot as plt

In [ ]:
dict_sub_tree = {}

for term in pvals_index:
    if graph[term]["namespace"] == "biological_process":
        dict_sub_tree.setdefault(graph[term]["namespace"], []).append(pvals_sorted[pvals_sorted.loc[:,'index'] == term].description_red.values[0])
    elif graph[term]["namespace"] == "cellular_component":
        dict_sub_tree.setdefault(graph[term]["namespace"], []).append(pvals_sorted[pvals_sorted.loc[:,'index'] == term].description_red.values[0])
    else:
        dict_sub_tree.setdefault(graph[term]["namespace"], []).append(pvals_sorted[pvals_sorted.loc[:,'index'] == term].description_red.values[0])

In [ ]:
class SimpleGroupedColorFunc(object):
    """Create a color function object which assigns EXACT colors
       to certain words based on the color to words mapping

       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.

       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def __init__(self, color_to_words, default_color):
        self.word_to_color = {word: color
                              for (color, words) in color_to_words.items()
                              for word in words}

        self.default_color = default_color

    def __call__(self, word, **kwargs):
        return self.word_to_color.get(word, self.default_color)

In [ ]:
from PIL import Image

In [ ]:
font_path = path.join('data', 'part_2', 'functional_analysis', 'Baskerville.ttf')

wc = WordCloud(
    font_path = font_path,
    mask = np.array(Image.open(path.join('data', 'part_2', 'functional_analysis', 'mask1.png'))),
    width = 800,
    height = 500,
    background_color="rgba(255, 255, 255, 0)",
    mode="RGBA",
    color_func = lambda *args, **kwargs: (18,10,143),
    max_words = 200,
    scale = 3,
    prefer_horizontal = 1,
    font_step=1,
    min_font_size = 7,
    max_font_size = 40
)



frequencies = pvals_sorted.set_index('description_red').fold_increase
wc.generate_from_frequencies(frequencies)

# palette 1
# color_to_words = {
#     '#264653': dict_sub_tree['biological_process'],
#     '#E9C46A': dict_sub_tree['cellular_component'],
#     '#E76F51': dict_sub_tree['molecular_function']
# }

# palette 2
color_to_words = {
    '#2E6F95': dict_sub_tree['biological_process'],
    '#43AA8B': dict_sub_tree['cellular_component'],
    '#B7094C': dict_sub_tree['molecular_function']
}

# Words that are not in any of the color_to_words values
# will be colored with a grey single color function
default_color = 'grey' 

# Create a color function with single tone
grouped_color_func = SimpleGroupedColorFunc(color_to_words, default_color)

# Apply our color function
wc.recolor(color_func=grouped_color_func)

path_wc = path.join('data', 'part_2', 'functional_analysis', 'wc.png')
wc.to_file(path_wc)
wc.to_image()